In [1]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("../finalRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows

ar = em.read_csv_metadata("../finalArkives.csv", encoding="ISO-8859-1", key='id')


c:\users\aparn\anaconda\envs\my_first_env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [2]:
rl['block'] = 'yes'

In [3]:
ar['block'] = 'yes'

In [ ]:
ar.head()

In [4]:
#Block on bogus attribute so that we can actually do
#blocking on genus
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'block', 'block', 
                     l_output_attrs=list(rl.columns.values), 
                     r_output_attrs=list(ar.columns.values))
len(C0)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:55


12687745

In [5]:
C1 = ob.block_candset(C0, 'genus', 'genus')
len(C1)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:03:29


3850

In [6]:
len(C1)


#matched over just genus

3850

In [7]:
C1

,_id,ltable_id,rtable_id,ltable_Unnamed: 0,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,...,rtable_family,rtable_genus,rtable_size,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_tCount,rtable_cCount,rtable_block
25215,25215,1470,3,1470,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,...,Scincidae,Tiliqua,Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australiaâ??s approximate...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1.0,yes
75598,75598,4363,9,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,...,Ranidae,Pelophylax,Male length: c. 71 mm (2)Female length: c. 74 mm (2),The main threats to the Albanian water frog are the drainage of wetlands and agrochemical and in...,The Albanian water frog is listed on Appendix III of the Bern Convention which means that it sho...,pet;pollution;,protected;,2,1.0,yes
75600,75600,4365,9,4365,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,...,Ranidae,Pelophylax,Male length: c. 71 mm (2)Female length: c. 74 mm (2),The main threats to the Albanian water frog are the drainage of wetlands and agrochemical and in...,The Albanian water frog is listed on Appendix III of the Bern Convention which means that it sho...,pet;pollution;,protected;,2,1.0,yes
78778,78778,7543,9,7543,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,...,Ranidae,Pelophylax,Male length: c. 71 mm (2)Female length: c. 74 mm (2),The main threats to the Albanian water frog are the drainage of wetlands and agrochemical and in...,The Albanian water frog is listed on Appendix III of the Bern Convention which means that it sho...,pet;pollution;,protected;,2,1.0,yes
83513,83513,4363,10,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,...,Ranidae,Pelophylax,Male length: c. 71 mm (2)Female length: c. 74 mm (2),The main threats to the Albanian water frog are the drainage of wetlands and agrochemical and in...,The Albanian water frog is listed on Appendix III of the Bern Convention which means that it sho...,pet;pollution;,protected;,2,1.0,yes
83515,83515,4365,10,4365,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,...,Ranidae,Pelophylax,Male length: c. 71 mm (2)Female length: c. 74 mm (2),The main threats to the Albanian water frog are the drainage of wetlands and agrochemical and in...,The Albanian water frog is listed on Appendix III of the Bern Convention which means that it sho...,pet;pollution;,protected;,2,1.0,yes
86693,86693,7543,10,7543,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,...,Ranidae,Pelophylax,Male length: c. 71 mm (2)Female length: c. 74 mm (2),The main threats to the Albanian water frog are the drainage of wetlands and agrochemical and in...,The Albanian water frog is listed on Appendix III of the Bern Convention which means that it sho...,pet;pollution;,protected;,2,1.0,yes
105744,105744,2849,13,2849,"alexandria cycad, dune cycad",Encephalartos,Zamiaceae,Terrestrial,South Africa (Eastern Cape Province);,\r\r\n The major threat to this species is removal by collectors. This has been the main caus

In [8]:
em.to_csv_metadata(C1, './candidate_set.csv')


True

In [142]:
label = em.split_train_test(C1, train_proportion=0.1039)
gold = label['train']
DS = label['test']


#Automatically label the training data
labels = []
matches = 0;
nonmatches = 0;
for index, row in gold.iterrows():
    if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
        labels.append(1)
        matches = matches + 1
    else:
        labels.append(0)
        nonmatches = nonmatches + 1
        
gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

number of matches in training set: 51
number of nonmatches in training set: 349


In [143]:
gold


,_id,ltable_id,rtable_id,ltable_Unnamed: 0,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,...,rtable_genus,rtable_size,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_tCount,rtable_cCount,rtable_block,label
1376132,1376132,6837,173,6837,NaN,Mammillaria,Cactaceae,Terrestrial,Mexico (Tamaulipas);,\r\r\n This species is threatened by illegal collecting because of its very small population ...,...,Mammillaria,Stem diameter: 3 cm (2)Flower length: 2.5 cm (2),Collectors have prized this distinctive and unusual cactus over the years and well-known sites h...,International trade in plants and seeds of wild origin is banned by the listing of <i>Mammillari...,,cites;,0,1.0,yes,0
7281388,7281388,7503,919,7503,NaN,Eleutherodactylus,Eleutherodactylidae,Terrestrial,Cuba;,"\r\r\n The major threat is habitat destruction and deforestation, as a result of the impacts ...",...,Eleutherodactylus,Female snout-vent length: up to 34 mm (2),Currently almost a quarter of all amphibian species on the British Virgin Islands are categorise...,A number of conservation recommendations have been made to protect the native fauna of the Briti...,,protected;,0,1.0,yes,0
9071248,9071248,658,1146,658,NaN,Phelsuma,Gekkonidae,Terrestrial,Madagascar;,"\r\r\n The threats to this species are poorly documented, but it is largely reliant on bamboo...",...,Phelsuma,Snout-vent length: 3.3 - 4.8 cm (2),Occupying an area thought to be no more than nine square kilometres the day gecko is threatened ...,The Ambatotsirongorongo Forest which supports a large proportion of the remaining day gecko popu...,loss;pet;,protected;,2,1.0,yes,0
4960177,4960177,5387,626,5387,perriers sifaka,Propithecus,Indriidae,Terrestrial,Madagascar;,\r\r\n Slash-and-burn activities resulting in forest loss represents the greatest threat to t...,...,Propithecus,5 â?? 6.5 kg (2),<strong>Slash-and-burn</strong> agriculture a common practice in Madagascar alongside logging po...,Milne-Ewardsâ?? sifaka occurs in two national parks Andringitra and Ranomafana (1) although as m...,loss;hunting;,cites;protected;,2,2.0,yes,0
5666533,5666533,7308,715,7308,NaN,Nepenthes,Nepenthaceae,Terrestrial,Indonesia (Sumatera);,\r\r\n Over-collection of plants from the wild represents a significant threat to this specie...,...,Nepenthes,Height: up to 5m (2)Pitcher height: 20 cm (2),<i>Nepenthes</i> species are threatened by a combination of over-collection and habitat loss (2)...,Although listed under Appendix II of the Convention on International Trade in Endangered Species...,loss;environmental;environment;,cites;,3,1.0,yes,0
12298850,12298850,6855,1553,6855,NaN,Turbinicarpus,Cactaceae,Terrestrial,Mexico (San Luis PotosÃ­);,\r\r\n This cactus is threatened by illegal collecting in a portion of its range. The only kn...,...,Turbinicarpus,Diameter: up to 5.5 cm (2) (3),This cactus is known from an area of just 5 square kilometres and its population is estimated at...,<em>Turbinicarpus subterraneus</em> ssp. <em>booleanus </em>is listed on Appendix I of the Conve...,,cites;,0,1.0,yes,0
11150560,11150560,6240,1408,6240,"light red meranti, philippine mahogany, white lauan",Shorea,Dipterocarpaceae,Terrestrial,"Malaysia (Sabah, Sarawak); Philippines;",NaN,...,Shorea,Height: 70 m (2)Trunk diameter: 1.6 m (2),The Philippine mahogany has become depleted due to logging and <strong>slash-and-burn</strong> f...,Despite being Critically Endangered it is not clear whether any conservation measures exist for ...,,,0,0.0,yes,1
11150267,11150267,5947,1408,5947,NaN,Shorea,Dipterocarpaceae,Terrestrial,Sri Lanka;,\r\r\n Regeneration is somewhat sparse.\r\r\n\r\r\n \r\r\n,...,Shorea,Height: 70 m (2)Trunk diameter: 1.6 m (2),The Philippine mahogany has become depleted due to logging and <strong>slash-and-burn</strong> f...,Despite being Critically Endangered it is not clear whether any conservation measures exist for ...,,,0,0.0,yes,0
9071238,9071238,6

In [ ]:
em.to_csv_metadata(gold, './train.csv')
em.to_csv_metadata(DS, './test.csv')

In [146]:
test_train = em.split_train_test(gold, train_proportion=0.7)
train = test_train['train']
test = test_train['test']
len(train)

280

In [148]:
rl.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)
ar.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)

In [156]:
feat = em.get_features_for_matching(rl, ar)
#em._match_s
feat['feature_name']

0                                     Unnamed_Unnamed_exm
1                                     Unnamed_Unnamed_anm
2                                Unnamed_Unnamed_lev_dist
3                                 Unnamed_Unnamed_lev_sim
4                               name_name_jac_qgm_3_qgm_3
5                           name_name_cos_dlm_dc0_dlm_dc0
6                           name_name_jac_dlm_dc0_dlm_dc0
7                                           name_name_mel
8                                      name_name_lev_dist
9                                       name_name_lev_sim
10                                          name_name_nmw
11                                           name_name_sw
12                          family_family_jac_qgm_3_qgm_3
13                      family_family_cos_dlm_dc0_dlm_dc0
14                      family_family_jac_dlm_dc0_dlm_dc0
15                                      family_family_mel
16                                 family_family_lev_dist
17            

In [150]:
em._match_s

{'abs_norm': <function py_entitymatching.feature.simfunctions.abs_norm>,
 'affine': <function py_entitymatching.feature.simfunctions.affine>,
 'cosine': <function py_entitymatching.feature.simfunctions.cosine>,
 'dice': <function py_entitymatching.feature.simfunctions.dice>,
 'exact_match': <function py_entitymatching.feature.simfunctions.exact_match>,
 'hamming_dist': <function py_entitymatching.feature.simfunctions.hamming_dist>,
 'hamming_sim': <function py_entitymatching.feature.simfunctions.hamming_sim>,
 'jaccard': <function py_entitymatching.feature.simfunctions.jaccard>,
 'jaro': <function py_entitymatching.feature.simfunctions.jaro>,
 'jaro_winkler': <function py_entitymatching.feature.simfunctions.jaro_winkler>,
 'lev_dist': <function py_entitymatching.feature.simfunctions.lev_dist>,
 'lev_sim': <function py_entitymatching.feature.simfunctions.lev_sim>,
 'monge_elkan': <function py_entitymatching.feature.simfunctions.monge_elkan>,
 'needleman_wunsch': <function py_entitymatch

In [151]:
em._match_c['corres']

[('Unnamed', 'Unnamed'),
 ('id', 'id'),
 ('name', 'name'),
 ('genus', 'genus'),
 ('family', 'family'),
 ('scientific_name', 'scientific_name'),
 ('block', 'block')]

In [152]:
feat_sub1 = feat[4:12]
feat_sub1

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
4,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x00000165A53FEE18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x00000165A53FEBF8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x00000165A53FE6A8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x00000165A53FEB70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,name_name_lev_dist,name,name,None,None,lev_dist,<function name_name_lev_dist at 0x00000165A53FEC80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,name_name_lev_sim,name,name,None,None,lev_sim,<function name_name_lev_sim at 0x00000165A53FE488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
10,name_name_nmw,name,name,None,None,needleman_wunsch,<function name_name_nmw at 0x00000165A53FED08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
11,name_name_sw,name,name,None,None,smith_waterman,<function name_name_sw at 0x00000165A53FE400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [157]:
H = em.extract_feature_vecs(train, 
                            feature_table=feat, 
                            attrs_after='label',
                            show_progress=False) 

In [158]:
H


,_id,ltable_id,rtable_id,Unnamed_Unnamed_exm,Unnamed_Unnamed_anm,Unnamed_Unnamed_lev_dist,Unnamed_Unnamed_lev_sim,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,...,scientific_name_scientific_name_lev_sim,scientific_name_scientific_name_nmw,scientific_name_scientific_name_sw,block_block_lev_dist,block_block_lev_sim,block_block_jar,block_block_jwn,block_block_exm,block_block_jac_qgm_3_qgm_3,label
11476241,11476241,7406,1449,0,0.195652,3,0.25,NaN,NaN,NaN,...,0.590909,11.0,13.0,0.0,1.0,1.0,1.0,1,1.0,0
6249012,6249012,4077,789,0,0.193525,4,0.00,0.137931,0.408248,0.250000,...,0.631579,11.0,12.0,0.0,1.0,1.0,1.0,1,1.0,0
6617125,6617125,185,836,0,0.221292,3,0.00,NaN,NaN,NaN,...,0.571429,9.0,9.0,0.0,1.0,1.0,1.0,1,1.0,0
12265184,12265184,4849,1549,0,0.319447,2,0.50,NaN,NaN,NaN,...,0.555556,7.0,8.0,0.0,1.0,1.0,1.0,1,1.0,0
6249037,6249037,4102,789,0,0.192345,4,0.00,0.172414,0.408248,0.250000,...,0.578947,11.0,11.0,0.0,1.0,1.0,1.0,1,1.0,0
11234187,11234187,2802,1419,0,0.506424,4,0.00,NaN,NaN,NaN,...,0.423077,5.0,10.0,0.0,1.0,1.0,1.0,1,1.0,0
2052862,2052862,2877,259,0,0.090024,3,0.25,NaN,NaN,NaN,...,0.437500,4.0,6.0,0.0,1.0,1.0,1.0,1,1.0,0
7273474,7273474,7504,918,0,0.122335,4,0.00,NaN,NaN,NaN,...,0.703704,19.0,19.0,0.0,1.0,1.0,1.0,1,1.0,0
6178118,6178118,4418,780,0,0.176550,4,0.00,0.521739,0.666667,0.500000,...,0.714286,14.0,15.0,0.0,1.0,1.0,1.0,1,1.0,0
4567145,4567145,190,577,0,0.329289,3,0.00,0.282051,0.577350,0.400000,...,0.578947,10.0,11.0,0.0,1.0,1.0,1.0,1,1.0,0
